# Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import random

# Tree Class

In [16]:
class Tree:
    def __init__(self):
        """constructor"""
        self.__state=0

    @property
    def getState(self):
        return self.__state
    
    
    def setState(self,value):
        self.__state=value
    
    def __str__(self):
        """returns a symbol for displaying a tree on the console"""
        return ".TBF"[self.__state]
    
    def is_neighbour_burning_tree(self,is_burning):
        """
        Updates the tree's state based on the presence of a burning tree in the neighborhood.
        :param is_burning: Boolean indicating if there is a burning tree nearby.
        """
        if is_burning==True and self.getState==1:#tree is healthy
            self.setState(2) #catches fire
            
        elif self.getState==2: #tree is burning
            self.setState(3) #tree is burned

# Forest Class

In [17]:
class Forest:
    def __init__(self,rows,cols,probability=None):
        """Constructor for the forest grid."""
        self.__n_rows=rows
        self.__n_cols=cols
        self.__p=probability
        self.__grid=[ [Tree() for _ in range(self.__n_cols)] for _ in range(self.__n_rows)]        
        #random initialisation of the grid.
        if self.probability is not None:
            self.init_grid(self.probability)
    
    def init_grid(self,probability):
        """Initializes the grid with healthy trees based on the probability."""
        for i in range(self.rows):
            for j in range(self.cols):
                if random.random() < self.probability:
                    # With probability p, the cell starts with a healthy tree
                    self.getCell(i,j).setState(1)
        
    @property
    def rows(self):
        return self.__n_rows
    @property
    def cols(self):
        return self.__n_cols
    @property
    def probability(self):
        return self.__p
    
    def getGrid(self):
        return self.__grid
    
    def getCell(self,i,j):
        return self.__grid[i][j]
    
    def __str__(self):
        #print the grid
        return '\n'.join(' '.join(str(tree) for tree in row) for row in self.__grid)
    
    def set_cell_on_fire(self,i,j):
        #we check if the parameter is in the range
        if (i >=0 or i< self.rows) and (j >=0 or j<self.cols):
            self.__grid[i][j].setState(2) # we set this tree on fire.
    
    def is_cell_neighbour_burning_tree(self,i,j):
        """checks if one of the neighbours of the cell is a burning tree. Returns True if it is."""
        #here are the possible neighbours : up, down left, right 
        neighbours = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        
        # we check each neighbour:
        for n_i, n_j in neighbours:
            #we only verify the neighbours within the boarders of the grid:
            pos_i=i + n_i
            pos_j=j+n_j
            if (pos_i >= 0) and (pos_i < self.rows)and ( (pos_j >=0) and (pos_j<self.cols)):
                #the cell is within the matrix, so we check if it is a burning tree.
                if self.getCell(pos_i,pos_j).getState==2:
                    return True
        
        return False # no burning tree was found on any neighbour.
    
    
    def update_grid(self):
        # Create a deep copy of the grid to store the updates
        new_grid = copy.deepcopy(self.__grid)
        for i in range(self.rows):
            for j in range(self.cols):
                has_burning_neighbor = self.is_cell_neighbour_burning_tree(i, j)
                new_grid[i][j].is_neighbour_burning_tree(has_burning_neighbor)
        self.__grid = new_grid

    def tree_proportion(self):
        number_healthy_trees=0
        
        #we get the number of healthy trees
        for i in range(self.rows):
            for j in range(self.cols):
                if self.__grid[i][j].getState==1:
                    number_healthy_trees=number_healthy_trees+1
        #get the number of cells of the grid
        number_of_cells=self.rows*self.cols
        return f'proportion of trees {number_healthy_trees/number_of_cells}'
        
    
    def generation_run(self,n_generation):
        n=abs(n_generation) #we make him strictly positive.
        
        #set a random tree on fire. We make sure that we select a healthy tree and not an empty cell.
        healthy_trees = [(i, j) for i in range(self.rows) for j in range(self.cols) if self.getCell(i, j).getState == 1]
        if healthy_trees:
            rand_i, rand_j = random.choice(healthy_trees)
            self.set_cell_on_fire(rand_i, rand_j)
        
        for i in range(n):
            # Skip the first generation as we have already set a tree on fire.
            if i != 0:  
                #we save the old state 
                old_grid = [[tree.getState for tree in row] for row in self.getGrid()]
                self.update_grid()
                #we save the new state
                new_grid = [[tree.getState for tree in row] for row in self.getGrid()]

                # Check if the grid state has changed.
                if old_grid == new_grid:
                    break

            print(f'Generation {i + 1}:')
            print(self)
            print(self.tree_proportion())
            print("")
        pass

In [18]:
healthy_tree_positions = [
    (0, 1), (0, 3), (0, 5), (0, 6), (0, 7),
    (1, 0), (1, 2), (1, 3), (1, 6),
    (2, 2), (2, 3), (2, 5), (2, 6),(2, 8),
    (3, 1), (3, 2), (3, 3), (3, 5),(3, 7),
    (4, 0),(4, 1), (4, 2),(4, 4), (4, 5), (4, 6), (4, 7), (4, 9),
    (5, 1), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9),
    (6, 0), (6, 1), (6, 1),(6, 5),(6, 7), 
    (7, 0),(7, 1), (7, 2), (7, 3),(7, 4), (7, 5), (7, 6), (7, 9),
    
    (8, 1),(8, 4), (8, 5), (8, 7), (8, 8), (8, 9),
    (9, 0), (9, 2), (9, 9), (9, 6), (9, 7), (9, 8)
]


forest = Forest(10, 10,0.2)

# Set the healthy trees.
for position in healthy_tree_positions:
    i, j = position
    forest.getCell(i,j).setState(1)
    

forest.generation_run(10)

Generation 1:
. T T T . T T T T T
T . T T T B T . . .
. . T T . T T . T T
T T T T . T . T T T
T T T T T T T T . T
. T . . T T T T T T
T T T T . T . T . .
T T T T T T T T . T
. T T . T T . T T T
T . T . . . T T T T
proportion of trees 0.71

Generation 2:
. T T T . B T T T T
T . T T B F B . . .
. . T T . B T . T T
T T T T . T . T T T
T T T T T T T T . T
. T . . T T T T T T
T T T T . T . T . .
T T T T T T T T . T
. T T . T T . T T T
T . T . . . T T T T
proportion of trees 0.67

Generation 3:
. T T T . F B T T T
T . T B F F F . . .
. . T T . F B . T T
T T T T . B . T T T
T T T T T T T T . T
. T . . T T T T T T
T T T T . T . T . .
T T T T T T T T . T
. T T . T T . T T T
T . T . . . T T T T
proportion of trees 0.63

Generation 4:
. T T B . F F B T T
T . B F F F F . . .
. . T B . F F . T T
T T T T . F . T T T
T T T T T B T T . T
. T . . T T T T T T
T T T T . T . T . .
T T T T T T T T . T
. T T . T T . T T T
T . T . . . T T T T
proportion of trees 0.58

Generation 5:
. T B F . F F F B T
T . F 

In [19]:
healthy_tree_positions2 = [
                      (0,5),
    (1,1),(1,3),(1,4),(1,5),
          (2,3),
    (3,1),(3,3),(3,4),      (3,6),
    (4,1),(4,2),(4,4),      (4,6),
                      (5,5),(5,6),
    
    (7,1),      (7,4),       (7,6),(7,7)
    
]
forest2 = Forest(8, 8,0.5)

# Set the healthy trees.
for position in healthy_tree_positions2:
    i, j = position
    forest2.getCell(i,j).setState(1)

forest2.generation_run(100)

Generation 1:
T . . . T T . T
. T . B T T T T
T T . T T . . .
T T T T T T T T
. T T T T T T T
T . . T . T T .
. . T T T T . T
. T . . T T T T
proportion of trees 0.65625

Generation 2:
T . . . T T . T
. T . F B T T T
T T . B T . . .
T T T T T T T T
. T T T T T T T
T . . T . T T .
. . T T T T . T
. T . . T T T T
proportion of trees 0.625

Generation 3:
T . . . B T . T
. T . F F B T T
T T . F B . . .
T T T B T T T T
. T T T T T T T
T . . T . T T .
. . T T T T . T
. T . . T T T T
proportion of trees 0.5625

Generation 4:
T . . . F B . T
. T . F F F B T
T T . F F . . .
T T B F B T T T
. T T B T T T T
T . . T . T T .
. . T T T T . T
. T . . T T T T
proportion of trees 0.484375

Generation 5:
T . . . F F . T
. T . F F F F B
T T . F F . . .
T B F F F B T T
. T B F B T T T
T . . B . T T .
. . T T T T . T
. T . . T T T T
proportion of trees 0.390625

Generation 6:
T . . . F F . B
. T . F F F F F
T B . F F . . .
B F F F F F B T
. B F F F B T T
T . . F . T T .
. . T B T T . T
. T . . T T T T
prop

In [20]:
forest = Forest(10, 10, 0.4)
forest.generation_run(10)

Generation 1:
. . T . T T . T . T
. T T . . . T . T T
T T T . . . . T . T
. T . T T . . T . .
T . . . T T T T . T
T . . T T . . T . .
. . . . . . . . . .
. T T T . T T . T T
. . . T . T . . . T
T . T B T T T T T .
proportion of trees 0.46

Generation 2:
. . T . T T . T . T
. T T . . . T . T T
T T T . . . . T . T
. T . T T . . T . .
T . . . T T T T . T
T . . T T . . T . .
. . . . . . . . . .
. T T T . T T . T T
. . . B . T . . . T
T . B F B T T T T .
proportion of trees 0.43

Generation 3:
. . T . T T . T . T
. T T . . . T . T T
T T T . . . . T . T
. T . T T . . T . .
T . . . T T T T . T
T . . T T . . T . .
. . . . . . . . . .
. T T B . T T . T T
. . . F . T . . . T
T . F F F B T T T .
proportion of trees 0.41

Generation 4:
. . T . T T . T . T
. T T . . . T . T T
T T T . . . . T . T
. T . T T . . T . .
T . . . T T T T . T
T . . T T . . T . .
. . . . . . . . . .
. T B F . T T . T T
. . . F . B . . . T
T . F F F F B T T .
proportion of trees 0.38

Generation 5:
. . T . T T . T . T
. T T 

# ToricForest

In [21]:
class ToricForest(Forest):
    def __init__(self,rows,cols,probability=None):
        """it is a forest, so it will inherit from his base structure"""
        super().__init__(rows, cols, probability)
        
    def is_cell_neighbour_burning_tree(self,i,j):
        """we change the functionality of the method. Since it is a toric forest it means that it has no end to cols and rows
            the concept remains the same, except that when we reach the boarder, we start on the other side of the matrix.
            Which means if we are on the right border, we check the first left row elements.
        """
        #here are the possible neighbours : up, down left, right 
        neighbours = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        
        # we check each neighbour:
        for n_i, n_j in neighbours:
            #we now must manage the fact if they exceed the boarder they go to the other side.
            pos_i=i + n_i
            pos_j=j+ n_j
            
            #in this case we adapt the pos_i and pos_j based on their position.

            if pos_i == self.rows: #we are at the lowest border, the neighbour is on the highest row.
                pos_i=0
            elif pos_i <0 : #we are at the highest border, the neighbour is on the lowest row.
                pos_i=self.rows-1
            
            if pos_j == self.cols: #we are at the right border, the neighbour is on the leftest column.
                pos_j=0
            elif pos_j <0 : #we are at the left border, the neighbour is on the rightest column.
                pos_i=self.cols-1            
            
            #we check if it is a burning tree.
            if self.getCell(pos_i,pos_j).getState==2:
                return True


tf=ToricForest(10,10,0.4)
print(tf)

T T . . T . . T . .
T . T T . T . . . .
. . . . T . T . T T
. . . T . . . T . .
T . . . . . T . . .
. . T . . . . . . .
. T . . T T . T T .
. T T . T . T T . .
. . T . T . T . T .
. T . T . T . . . .


In [26]:
tf=ToricForest(10,10,0.4)
tf.generation_run(100)

Generation 1:
T T . . . . T . T T
. T T . T . . . . .
T . T T T . T . . .
. . T . T . . T . .
T . . T . . . T T .
. T B . . . T T . .
T T . . T T . T T T
T . . . . . . . T T
. . . . T . T . . .
T T T T T . . . T T
proportion of trees 0.42

Generation 2:
T T . . . . T . T T
. T T . T . . . . .
T . T T T . T . . .
. . T . T . . T . .
T . . T . . . T T .
. B F . . . T T . .
T T . . T T . T T T
T . . . . . . . T T
. . . . T . T . . .
T T T T T . . . T T
proportion of trees 0.41

Generation 3:
T T . . . . T . T T
. T T . T . . . . .
T . T T T . T . . .
. . T . T . . T . .
T . . T . . . T T .
. F F . . . T T . .
T B . . T T . T T T
T . . . . . . . T T
. . . . T . T . . .
T T T T T . . . T T
proportion of trees 0.4

Generation 4:
T T . . . . T . T T
. T T . T . . . . .
T . T T T . T . . .
. . T . T . . T . .
T . . T . . . T T .
. F F . . . T T . .
B F . . T T . T T T
T . . . . . . . T T
. . . . T . T . . .
T T T T T . . . T T
proportion of trees 0.39

Generation 5:
T T . . . . T . T T
. T T .